In [8]:
# Mean Reciprocal Rank

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import os

if "DATABRICKS_RUNTIME_VERSION" in os.environ and not 'installed_libs' in globals():
  #CUDA = 'cu121' 
  installed_libs = True
  
  
  !pip install torch==2.1.0  torchvision==0.16.0 torchtext==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121
  import torch
  #os.environ['TORCH'] = torch.__version__
  #print(torch.__version__)
  #torch_version = '2.0.0+cu118'
  
  #!pip install pyg_lib torch_scatter torch_sparse torch_cluster -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html # torch_spline_conv
  !pip install torch_geometric
  !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html
  #!pip install torch_sparse -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  #!pip install torch_scatter -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  #!pip install pyg_lib -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  !pip install sentence-transformers
  !pip install torcheval
  !pip install matplotlib
  !pip install pandas
  !pip install tensorboard
  !pip install torchfm
  
if "DATABRICKS_RUNTIME_VERSION" in os.environ:
  ROOT_FOLDER = '/dbfs/FileStore/GraphNeuralNetworks/'
else:
  ROOT_FOLDER = ''

In [11]:
# sampler

def nf_sampler(batch_size, neg_sample_ratio, edge_label_index, edge_label, num_learnings):
    # triplet mode only
    # sample some random edges
    edge_label_index1 = edge_label_index[edge_label==1]
    num_samples = batch_size
    #sampled_indices = torch.randint(0, edge_label_index.shape[1], (num_samples,), replacement=False)
    sampled_indices = torch.randperm(edge_label_index1.shape[1])[:num_samples]
    sampled_edges = edge_label_index1[:, sampled_indices]
    # remove sampled edges from edge_label_index with mask
    mask = torch.ones(edge_label_index1.shape[1], dtype=torch.bool)
    mask[sampled_indices] = False
    edge_label_index1 = edge_label_index1[:, mask]
    
    assert neg_sample_ratio >= 1
    neg_samples = sampled_indices.shape[0]*neg_sample_ratio
    
    s = sampled_edges[0,:].unsqueeze(0)
    src_edges = s
    for i in range(neg_sample_ratio-1):
        src_edges= torch.cat((src_edges,s),dim=1)

    sampled_negatives = torch.randint(0, num_learnings, (neg_samples,)).squeeze().unsqueeze(0)
   
   
    negative_edge_label_index = torch.cat((src_edges, sampled_negatives),dim=0)
    edge_label_indices = torch.cat((sampled_edges, negative_edge_label_index),dim=1)
    new_edge_label_index = edge_label_index
    batch_edge_label_index = edge_label_indices
    return new_edge_label_index, batch_edge_label_index, torch.cat((torch.ones(batch_size), torch.zeros(neg_samples)))

def nf_loader(edge_label_index, batch_size, num_learnings, neg_sample_ratio):
    while edge_label_index.shape[1] > 0:
        new_edge_label_index, batch_edge_label_index, batch_labels = nf_sampler(batch_size, neg_sample_ratio, edge_label_index, num_learnings)
        edge_label_index = new_edge_label_index
        yield batch_edge_label_index, batch_labels
    

def get_total_minibatch_count_fm(batch_size, edge_label_index):
    return int((edge_label_index.shape[1]+batch_size)//batch_size)
import torch
from torch_geometric.data import HeteroData
#data = HeteroData(torch.load('factorization_machines_dataset.pt'))
train_data = HeteroData(torch.load(ROOT_FOLDER+'FactorizationMachines_Dataset_train_labelencoded_v2.pt'))
val_data = HeteroData(torch.load(ROOT_FOLDER+'FactorizationMachines_Dataset_val_labelencoded_v2.pt'))
test_data = HeteroData(torch.load(ROOT_FOLDER+'FactorizationMachines_Dataset_test_labelencoded_v2.pt'))

train_embeddings = HeteroData(torch.load(ROOT_FOLDER+'FactorizationMachines_Dataset_train_v1.pt'))
val_embeddings = HeteroData(torch.load(ROOT_FOLDER+'FactorizationMachines_Dataset_val_v1.pt'))
val_embeddings = HeteroData(torch.load(ROOT_FOLDER+'FactorizationMachines_Dataset_val_v1.pt'))
test_embeddings = HeteroData(torch.load(ROOT_FOLDER+'FactorizationMachines_Dataset_val_v1.pt'))

In [12]:
import torch
import torch.nn.functional as F
from models.TransE import TransE
from models.DistMult import DistMult
from models.FactorizationMachineModel import FactorizationMachineModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Model(torch.nn.Module):
    def __init__(self, fm : torch.nn.Module, head, node_types, edge_types, ggn_output_dim, pnorm=1, num_supervisors=0, num_organizations=0):
        super().__init__()
        # edge_type onehot lookup table with keys
        # node_type onehot lookup table with keys
        self.node_type_embedding = torch.nn.Embedding(len(node_types), ggn_output_dim) # hidden channels should be the output dim of gnn
        self.num_supervisors = num_supervisors
        self.num_organizations = num_organizations
        self.edge_types = edge_types
        for edge_type in edge_types:
            if edge_type[1].startswith('rev_'):
                self.edge_types.remove(edge_type)
        
        # create edge to int mapping
        self.edgeindex_lookup = {edge_type:torch.tensor(i)  for i, edge_type in enumerate(edge_types)}
            
        if head=='TransE': 
            self.head = TransE(len(node_types), len(edge_types) , ggn_output_dim, p_norm= pnorm)  # KGE head with loss function
        elif head=='DistMult':
            self.head = DistMult(len(node_types), len(edge_types) , ggn_output_dim, p_norm= pnorm)  # KGE head with loss function
        else:
            raise NotImplementedError
        
        self.fm = fm
       
     
        self.layer1 = torch.nn.Linear(3167,512)
        self.layer2 = torch.nn.Linear(512,256)
        self.layer3 = torch.nn.Linear(257,256)
        self.fc_output = torch.nn.Linear(256, 256)

    def forward(self, hetero_data, hetero_data_embeddings, edge_label_index, edge_label):
        
    
        people = hetero_data['people'].x[edge_label_index[0,:],:]
        # last two columns in people are the indices of onehot, so change them to full onehot supervisor and organization
        #supervisors = torch.nn.functional.one_hot(people[:,-2].to(torch.int64), num_classes=self.num_supervisors).to(torch.float32)
        #organizations = torch.nn.functional.one_hot(people[:,-1].to(torch.int64), num_classes=self.num_organizations).to(torch.float32)
        #people = torch.cat((people[:,:-2], supervisors, organizations), dim=1)
        people_embeddings = hetero_data_embeddings['people'].x[edge_label_index[0,:],:]
                        
        learnings = hetero_data['courses_and_programs'].x[edge_label_index[1,:],:]
        learning_embeddings = hetero_data_embeddings['courses_and_programs'].x[edge_label_index[1,:],:]
        
        x1 = torch.cat((people_embeddings, learning_embeddings),dim=1)
        x1 = self.layer1(x1).relu()
        x1 = self.layer2(x1).relu()
        x2 = torch.cat((people,learnings),dim=1)
        x2 = self.fm(x2)
        #scores = x2
        x3 = self.layer3(torch.cat((x1,x2.unsqueeze(1)),dim=1)).relu()
        scores = self.fc_output(x3).relu()
        return scores
        pos_scores = scores[edge_label==1]
        neg_scores = scores[edge_label==0]
            
        
        return F.margin_ranking_loss(
            pos_scores,
            neg_scores,
            target=torch.ones_like(pos_scores), # 1 for similarity, -1 for dissimilarity
            margin=0.5
        )
        
    
out_channels = 1
hidden_channels = 16
num_heads = 0
num_layers = 0
pnorm = 2
head = 'TransE'
#gnn = HGT(hidden_channels=out_channels, out_channels=out_channels, num_heads=num_heads, num_layers=num_layers, node_types=train_data.node_types, data_metadata=metadata)
filename = 'HeteroData_Learnings_normalized_triangles_withadditionaldata_v1.pt'
data_forlookup = HeteroData.from_dict(torch.load(ROOT_FOLDER+filename))
num_supervisors = data_forlookup['people'].num_nodes
num_organizations = data_forlookup['organizations'].num_nodes
metadata = data_forlookup.metadata()
# add selfloops
for node_type in data_forlookup.node_types:
    metadata[1].append((node_type, 'self_loop', node_type))  
    
    

del data_forlookup
print(train_data['people'].labelencoding.shape, train_data['courses_and_programs'].labelencoding.shape)  
field_dims = torch.cat((train_data['people'].labelencoding,train_data['courses_and_programs'].labelencoding), dim=0)
print(field_dims)
# convert the field dims to integer
field_dims = field_dims.to(torch.int64)
fm = FactorizationMachineModel(
    field_dims=field_dims,
        embed_dim=hidden_channels)





fm_and_dense_folderfile = 'models/learningpeople_factorizationmachines_densemargin05aee_20231104_161222_pnorm2_llr2e-05_bs32_neighbors__head_TransE_hiddenchannels_16_outchannels_1_numheads_0_numlayers_0/model_samplesseen713728.pt'


modelfmdense = Model(fm, head=head, node_types=metadata[0], edge_types=metadata[1], ggn_output_dim=out_channels, pnorm=pnorm, num_supervisors=num_supervisors, num_organizations=num_organizations)
#torch_geometric.compile(model, dynamic=True)
modelfmdense.to(device)



torch.Size([56]) torch.Size([26])
tensor([293444,  55638, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 293444, 293444,
         13613,  13613,  55796, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052])


Model(
  (node_type_embedding): Embedding(6, 1)
  (head): TransE(6, num_relations=22, hidden_channels=1)
  (fm): FactorizationMachineModel(
    (embedding): FeaturesEmbedding(
      (embedding): Embedding(20822892, 16)
    )
    (linear): FeaturesLinear(
      (fc): Embedding(20822892, 1)
    )
    (fm): FactorizationMachine()
  )
  (layer1): Linear(in_features=3167, out_features=512, bias=True)
  (layer2): Linear(in_features=512, out_features=256, bias=True)
  (layer3): Linear(in_features=257, out_features=256, bias=True)
  (fc_output): Linear(in_features=256, out_features=256, bias=True)
)

In [13]:
### init model
from tqdm.auto import tqdm

train_data = train_data.to(device)
val_data = val_data.to(device)
train_embeddings = train_embeddings.to(device)
val_embeddings = val_embeddings.to(device)
modelfmdense.train()
start_epoch = 1
batch_size = 32


train_loader = nf_loader(train_data['people','completed','courses_and_programs'].edge_label_index.to('cpu'), train_data['people','completed','courses_and_programs'].edge_label.to('cpu'), batch_size, num_learnings, neg_sample_ratio)
val_loader = nf_loader(val_data['people','completed','courses_and_programs'].edge_label_index.to('cpu'),val_data['people','completed','courses_and_programs'].edge_label.to('cpu'), batch_size, num_learnings, neg_sample_ratio)
print('epoch1')
modelfmdense.eval()
modelfmdense.to(device)
for i, (batch_edge_label_index, labels) in tqdm(enumerate(train_loader), total=1):
    if not batch_edge_label_index.shape[1] != labels.shape[0]:
        
        loss = modelfmdense(train_data, train_embeddings, batch_edge_label_index.to(device), labels.to(device))
        break
modelfmdense.to('cpu')  

AttributeError: 'EdgeStorage' object has no attribute 'edge_label'

In [14]:
train_data['people','completed','courses_and_programs']

{'edge_index': tensor([[ 47782, 110396, 140611,  ..., 230612, 245078, 150101],
        [  8108,  38872,  25608,  ...,  25609,  28551,  25046]],
       device='cuda:0'), 'edge_label_index': tensor([[237571, 232648, 223362,  ..., 136880, 291335, 243006],
        [ 33929,  15584,  28839,  ...,  25372,  34527,  27229]],
       device='cuda:0')}

In [ ]:
modelfmdense.load_state_dict(torch.load(fm_and_dense_folderfile)['model_state_dict'])

In [ ]:
import torch
import torch.nn.functional as F
from models.TransE import TransE
from models.DistMult import DistMult
from models.FactorizationMachineModel import FactorizationMachineModel
import torch_geometric
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Model(torch.nn.Module):
    def __init__(self, fm : torch.nn.Module, head, node_types, edge_types, ggn_output_dim, pnorm=1, num_supervisors=0, num_organizations=0):
        super().__init__()
        # edge_type onehot lookup table with keys
        # node_type onehot lookup table with keys
        self.node_type_embedding = torch.nn.Embedding(len(node_types), ggn_output_dim) # hidden channels should be the output dim of gnn
        self.num_supervisors = num_supervisors
        self.num_organizations = num_organizations
        self.edge_types = edge_types
        for edge_type in edge_types:
            if edge_type[1].startswith('rev_'):
                self.edge_types.remove(edge_type)
        
        # create edge to int mapping
        self.edgeindex_lookup = {edge_type:torch.tensor(i)  for i, edge_type in enumerate(edge_types)}
            
        if head=='TransE': 
            self.head = TransE(len(node_types), len(edge_types) , ggn_output_dim, p_norm= pnorm)  # KGE head with loss function
        elif head=='DistMult':
            self.head = DistMult(len(node_types), len(edge_types) , ggn_output_dim, p_norm= pnorm)  # KGE head with loss function
        else:
            raise NotImplementedError
        
        self.fm = fm
       
     
        self.layer1 = torch.nn.Linear(3167,512)
        self.layer2 = torch.nn.Linear(512,256)
        self.layer3 = torch.nn.Linear(257,256)
        self.fc_output = torch.nn.Linear(256, 256)

    def forward(self, hetero_data, hetero_data_embeddings, edge_label_index, edge_label):
        
    
        people = hetero_data['people'].x[edge_label_index[0,:],:]
        # last two columns in people are the indices of onehot, so change them to full onehot supervisor and organization
        #supervisors = torch.nn.functional.one_hot(people[:,-2].to(torch.int64), num_classes=self.num_supervisors).to(torch.float32)
        #organizations = torch.nn.functional.one_hot(people[:,-1].to(torch.int64), num_classes=self.num_organizations).to(torch.float32)
        #people = torch.cat((people[:,:-2], supervisors, organizations), dim=1)
        #people_embeddings = hetero_data_embeddings['people'].x[edge_label_index[0,:],:]
                        
        learnings = hetero_data['courses_and_programs'].x[edge_label_index[1,:],:]
        #learning_embeddings = hetero_data_embeddings['courses_and_programs'].x[edge_label_index[1,:],:]
        
        #x1 = torch.cat((people_embeddings, learning_embeddings),dim=1)
        #x1 = self.layer1(x1).relu()
        #x1 = self.layer2(x1).relu()
        x2 = torch.cat((people,learnings),dim=1)
        x2 = self.fm(x2)
        scores = x2
        #x3 = self.layer3(torch.cat((x1,x2.unsqueeze(1)),dim=1)).relu()
        #scores = self.fc_output(x3).relu()
        return scores
        pos_scores = scores[edge_label==1]
        neg_scores = scores[edge_label==0]
        

        return F.margin_ranking_loss(
            pos_scores,
            neg_scores,
            target=torch.ones_like(pos_scores), # 1 for similarity, -1 for dissimilarity
            margin=0.5
        )
        
    
out_channels = 1
hidden_channels = 16
num_heads = 0
num_layers = 0
pnorm = 2
head = 'TransE'
#gnn = HGT(hidden_channels=out_channels, out_channels=out_channels, num_heads=num_heads, num_layers=num_layers, node_types=train_data.node_types, data_metadata=metadata)
filename = 'HeteroData_Learnings_normalized_triangles_withadditionaldata_v1.pt'
data_forlookup = HeteroData.from_dict(torch.load(ROOT_FOLDER+filename))
num_supervisors = data_forlookup['people'].num_nodes
num_organizations = data_forlookup['organizations'].num_nodes
metadata = data_forlookup.metadata()
# add selfloops
for node_type in data_forlookup.node_types:
    metadata[1].append((node_type, 'self_loop', node_type))  
    
    

del data_forlookup
print(train_data['people'].labelencoding.shape, train_data['courses_and_programs'].labelencoding.shape)  
field_dims = torch.cat((train_data['people'].labelencoding,train_data['courses_and_programs'].labelencoding), dim=0)
print(field_dims)
# convert the field dims to integer
field_dims = field_dims.to(torch.int64)
fm = FactorizationMachineModel(
    field_dims=field_dims,
        embed_dim=hidden_channels)





fm_only_folderfile = 'modelsmodels/learningpeople_factorizationmachines_densemargin05AAAFMONLY_20231104_172955_pnorm2_llr2e-05_bs32_neighbors__head_TransE_hiddenchannels_16_outchannels_1_numheads_0_numlayers_0/model_samplesseen1267456.pt'


modelfmonly = Model(fm, head=head, node_types=metadata[0], edge_types=metadata[1], ggn_output_dim=out_channels, pnorm=pnorm, num_supervisors=num_supervisors, num_organizations=num_organizations)
#torch_geometric.compile(model, dynamic=True)
modelfmonly.to(device)



In [ ]:
# evaluate model on test set
# init dimensions of model by training it
from tqdm.auto import tqdm
from datetime import datetime
import numpy as np 

def evaluate(model, n_negatives, model_folder, on='test'):
    num_neighbors = [int(x) for x in model_folder.split('neighbors_')[1].split('head')[0].strip('_').split('_')]
    
    model.to(device)
    mrrs = []
   
    
    #test_sampler = get_hgt_linkloader(test_data, input_edgetype, 1, False, 'triplet', n_negatives, num_neighbors, num_workers=0, prefetch_factor=None, pin_memory=True)
    if on=='test':
        test_sampler = nf_loader(test_data['people','completed','courses_and_programs'].edge_label_index.to('cpu'),test_data['people','completed','courses_and_programs'].edge_label.to('cpu'), batch_size, num_learnings, neg_sample_ratio)
    elif on=='train':
        test_sampler = nf_loader(train_data['people','completed','courses_and_programs'].edge_label_index.to('cpu'),train_data['people','completed','courses_and_programs'].edge_label.to('cpu'), batch_size, num_learnings, neg_sample_ratio)
        
        
    # test data
    best_mrr, best_differences, best_src_nodes, best_dst_nodes = 0, None, None, None
    for i, (batch_edge_label_index, labels) in tqdm(enumerate(test_sampler)):
        if i==1000:
            break
        model.eval()
        if on == 'test':
            
            differences = model(test_data.to(device), batch_edge_label_index.to(device), labels.to(device))
        elif on == 'train':
            differences = model(train_data.to(device), batch_edge_label_index.to(device), labels.to(device))
        #optimizer.zero_grad()
        #loss.backward()
        edge_label = lables
        # define mrr with differences and labels
        # get rank of positive edge from tensor, positive edge is first in batch

        differences = -1* differences.cpu().detach().numpy()
        edge_label = edge_label.cpu().detach().numpy()
        rank = (differences < differences[0]).sum()

        # reciprocal
        mrr = 1/(rank+1)
        if mrr > best_mrr:
            best_mrr = mrr
            # best_differences = differences
            # best_src_nodes = global_src_nodes
            # best_dst_nodes = global_dst_nodes
            print('new best mrr', best_mrr)
            
    
        mrrs.append(mrr)

        
        if False:
            jobpeople = test_data_text['jobs', 'job_student', 'people']
            job_edge = jobpeople.edge_index[1,:] == global_src_nodes[0]
            
            
            if torch.sum(job_edge) != 0:
                job_idx = jobpeople.edge_index[0,:][job_edge]
                
            else:
                job_idx = torch.tensor([0])

            job_edge = jobpeople.edge_label_index[1,:] == global_src_nodes[0]
            if torch.sum(job_edge) != 0:
                job_idx2 = jobpeople.edge_label_index[0,:][job_edge]
                torch.cat((job_idx, job_idx2)).squeeze()
                
                
            
        
            print('==============')
            print('==============')
            print('==============')
            # (differences < differences[0])
            print('mrr',mrr)
            print('rank',rank)
            for idx in job_idx:
                jobtitle = test_data_text['jobs'].TITLE[idx.item()]
                print('job title', jobtitle)
                
            print('original', test_data_text['courses_and_programs'].TITLE[global_dst_nodes[0]])
            print('original', test_data_text['courses_and_programs'].DESCRIPTION[global_dst_nodes[0]])
            
            # first 10 higher ranked
            mask = torch.where(torch.tensor((differences < differences[0])))
            indices = torch.argsort(torch.tensor(differences[mask]))
            for i in range(10):
                index = indices[i]
                global_index = global_dst_nodes[mask][index]
                print('------')
                print(differences[mask][index])
                print(test_data_text['courses_and_programs'].TITLE[global_index])
                print(test_data_text['courses_and_programs'].DESCRIPTION[global_index])

    print('mean mrr',np.mean(mrrs))
    print('mean rank',1/np.mean(mrrs))
    # mean rank
    model.to('cpu')


In [ ]:

evaluate(modelfmdense, 10000, fm_and_dense_folderfile, on='test')
evaluate(modelfmonly, 10000, fm_only_folderfile, on='test')

NameError: name 'evaluate' is not defined

In [ ]:
a = set(test_data['courses_and_programs', 'course_and_programs_student', 'people'].edge_label_index[0,:].unique().tolist())
a2 = set(train_data['courses_and_programs', 'course_and_programs_student', 'people'].edge_label_index[0,:].unique().tolist())

In [ ]:
b = set(torch.cat((train_data['courses_and_programs', 'course_and_programs_student', 'people'].edge_index[0,:].unique(),train_data['courses_and_programs', 'course_and_programs_student', 'people'].edge_label_index[0,:].unique())).tolist())

In [ ]:
len(a.intersection(b))

9335

In [ ]:
# evaluate(model_3layereuclid, 10000, model_folder3layereuclid, on='test')
# evaluate(model_2layereuclid, 10000, model_folder2layereuclid, on='test')
evaluate(model_2layerp1, 10000, model_folder2layerp1, on='test')

size of dataset on disk:  2.279761238 gb
loading saved heterodata object
for skill job edges keep top k edges per job, k is  50
keep tensor(1208056) of total 16289586


1it [00:06,  6.70s/it]

new best mrr 0.0024752475247524753


3it [00:12,  3.57s/it]

new best mrr 0.05


4it [00:14,  3.11s/it]

new best mrr 1.0


1000it [45:27,  2.73s/it]


mean mrr 0.17139854320945072
mean rank 5.834355305914065


In [ ]:
evaluate(model_3layereuclid, 10000, model_folder3layereuclid, on='train')
evaluate(model_2layereuclid, 10000, model_folder2layereuclid, on='train')
evaluate(model_2layerp1, 10000, model_folder2layerp1, on='train')

size of dataset on disk:  2.279761238 gb
loading saved heterodata object
for skill job edges keep top k edges per job, k is  50
keep tensor(1208056) of total 16289586


1it [00:15, 15.19s/it]

new best mrr 8.13206473123526e-05


2it [00:23, 10.85s/it]

new best mrr 0.000500751126690035


...

new best mrr 0.1111111111111111


8it [00:20,  2.25s/it]

new best mrr 1.0


200it [10:11,  3.06s/it]


mean mrr 0.11901925778350261
mean rank 8.402001647657823


In [ ]:
evaluate(modele4, 50000, model_foldere4)

In [ ]:
evaluate(modele4euclidean, 50000, model_foldere4euclidean)

In [ ]:
evaluate(modele5, 50000, model_foldere5, on='train')

In [ ]:
evaluate(modele4, 50000, model_foldere4, on='train')

In [ ]:
evaluate(modele4euclidean, 30000, model_foldere4euclidean, on='train')